In [44]:
import mysql.connector as connection
import pandas as pd
import os
import csv
import logging as lg

class loggerClass:
    def __init__(self):
        logger = lg.getLogger()
        fhandler = lg.FileHandler(filename='MYSQL_Operation.log', mode='a')
        formatter = lg.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fhandler.setFormatter(formatter)
        if (logger.hasHandlers()):
            logger.handlers.clear()
        logger.addHandler(fhandler)
        logger.setLevel(lg.INFO)
     
    def logFunc(self,lg_type,content):
        if lg_type == "INFO":
            lg.info(str(content))
            lg.shutdown()
        elif lg_type == "ERROR":
            lg.error(str(content))
            lg.shutdown()
        else:
            lg.info(str(content))
            lg.shutdown()

class MySQLOperation:
    def __init__(self,hostname,username,password,db_name):
        self.__hostname = hostname
        self.__username = username
        self.__password = password
        self.__db_name = db_name
        self.__lg = loggerClass()
        
    def db_Connection(self):
        try:
            self.__lg.logFunc("info", f"connecting to mysql db Host:{self.__hostname},User:{self.__username}")
            return connection.connect(host=self.__hostname, user=self.__username, passwd=self.__password,use_pure=True)
        except Exception as e:
            self.__lg.logFunc('error', "Error while establishing connection with MYSQL DB:" + str(e))
            
    def db_Connection_withDB(self):
        try:
            self.__lg.logFunc("info", f"connecting to mysql db Host:{self.__hostname},User:{self.__username},Database:{self.__db_name}")
            return connection.connect(host=self.__hostname, user=self.__username, passwd=self.__password,
                                      database = self.__db_name,use_pure=True)
        except Exception as e:
            self.__lg.logFunc('error', "Error while establishing connection with MYSQL DB:" + str(e))

    def create_Database(self):
        try:
            db     = self.db_Connection()
            cursor = db.cursor()
            query  = "CREATE DATABASE " + str(self.__db_name)
            cursor.execute(query,db)
            self.__lg.logFunc("info", f"Database {self.__db_name} created")
            db.close()
            
        except Exception as e:
            db.close()
            self.__lg.logFunc('error', f"Error while creating Database {self.__db_name}" + str(e))
    
    def create_Table(self,table_name,table_definition):
        try:
            db     = self.db_Connection_withDB()
            cursor = db.cursor()
            table_def = ''
            for val in table_definition:
                table_def += val + ' ' + str(table_definition[val]) + ', '
            table_def = table_def[:-2]
            query = "CREATE TABLE IF NOT EXISTS {db_n}.{tb}({struc})".format(db_n=self.__db_name, tb=table_name
                                                                                 , struc=table_def)
            cursor.execute(query,db)
            self.__lg.logFunc("info", f"Table {table_name} created with query {query}")         
            
        except Exception as e:
            db.close()
            self.__lg.logFunc('error', f"Error while creating Table {table_name}" + str(e))
            
          
    def insert_Data(self,table_name,file_path,file_name):
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            file_location = file_path + '\\' + file_name
            with open(file_location,'r') as file:
                self.__lg.logFunc("info", f"File {file_location} opened") 
                fileRead = csv.reader(file,delimiter = '\n')
                row_data = [tuple(i[0].split(',')) for i in fileRead]
                query = "INSERT INTO {db}.{tab} VALUES ({val})".format(db=self.__db_name, tab=table_name,
                                                                                val='%s,%s,%s,%s,%s,%s,%s')
                try:
                    cursor.executemany(query,row_data)
                except Exception as e:
                    self.__lg.logFunc('error', f"Error while inserting data into {table_name}" + str(query))
                self.__lg.logFunc("info", f"File {file_location} data inserted into {table_name}; record count {len(row_data)}") 
                db.commit()
            file.close()
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while inserting data into {table_name}" + str(e))
            db.close()
            
    def check_Table_Data_Count(self,table_name):
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            query = f"SELECT COUNT(*) FROM {table_name}"
            cursor.execute(query,db)
            result = cursor.fetchone()[0]
            print(f"The {table_name} has {result} records")
            self.__lg.logFunc("info", f"Checking count of records from table {table_name}")
            self.__lg.logFunc("info", f"The {table_name} has {result} records")
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while checking {table_name} table data count" + str(e))
            db.close()
            
    def check_column_Data_Count(self,table_name,col_name):
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            query = f"SELECT {col_name},COUNT(*) FROM {table_name} group by {col_name}"
            cursor.execute(query,db)
            result = cursor.fetchall()
            for rec in result:
                print(rec)
            self.__lg.logFunc("info", f"The group count of {col_name} in {table_name} has been checked")
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while checking {table_name} table data count" + str(e))
            db.close()
            
    def filter_data(self,table_name,filter_criteria):
        """Give Filter_Criteria in dictionary format"""
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            condition = " AND ".join([key+"='"+str(filter_criteria[key])+"'" for key in filter_criteria.keys()])
            query = f"SELECT * FROM {table_name} where {condition}"
            cursor.execute(query,db)
            result = cursor.fetchall()
            print(str(len(result)) + ' records fetched')
            for rec in result:
                print(rec)
            self.__lg.logFunc("info", f"The filter criteria {filter_criteria} of {table_name} has been performed successfully")
            self.__lg.logFunc("info", str(len(result)) + ' records fetched')
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while checking {table_name} table data count" + str(e))
            db.close()
            
    def update_data(self,table_name,set_value,update_criteria):
        """Give Filter_Criteria in dictionary format"""
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            set_ = ",".join([key+"='"+str(set_value[key])+"'" for key in set_value.keys()])
            condition = " AND ".join([key+"='"+str(update_criteria[key])+"'" for key in update_criteria.keys()])
            query = f"UPDATE {self.__db_name}.{table_name} set {set_} where {condition}"
            cursor.execute(query,db)
            db.commit()
            self.__lg.logFunc("info", f"The update operation on {table_name} with set condition {set_} and filter condition {condition} has been performed successfully")
            self.__lg.logFunc("info", f"Total {cursor.rowcount} updated")
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while updating table {table_name}" + str(e))
            db.close()
            
    def delete_table(self,table_name):
        """Give Filter_Criteria in dictionary format"""
        try:
            db = self.db_Connection_withDB()
            cursor = db.cursor()
            query = f"DROP TABLE {table_name}"
            cursor.execute(query,db)
            self.__lg.logFunc("info", f"The table {table_name} has been deleted")
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while deleing table {table_name}" + str(e))
            db.close()
            
    def delete_database(self,db_name):
        """Give Filter_Criteria in dictionary format"""
        try:
            db = self.db_Connection()
            cursor = db.cursor()
            query = f"DROP DATABASE {db_name}"
            cursor.execute(query,db)
            self.__lg.logFunc("info", f"The database {db_name} has been deleted")
            db.close()
            
        except Exception as e:
            self.__lg.logFunc('error', f"Error while deleing database {table_name}" + str(e))
            db.close()

In [45]:
mydb = MySQLOperation("localhost","root","mysql","cardataset")

In [46]:
mydb.create_Database()

In [47]:
mydb.create_Table("car",{"buying":"varchar(10)","maint":"varchar(10)","doors":"varchar(10)","persons":"varchar(10)",
                            "lug_boot":"varchar(10)","safety":"varchar(10)","class":"varchar(10)"})

In [48]:
mydb.insert_Data("car","D:\python_1","car.data")

In [49]:
mydb.check_Table_Data_Count("car")

The car has 1728 records


In [50]:
mydb.check_column_Data_Count("car","buying")

('vhigh', 432)
('high', 432)
('med', 432)
('low', 432)


In [51]:
mydb.filter_data("car",{"buying":"high","doors":"3"})

108 records fetched
('high', 'vhigh', '3', '2', 'small', 'low', 'unacc')
('high', 'vhigh', '3', '2', 'small', 'med', 'unacc')
('high', 'vhigh', '3', '2', 'small', 'high', 'unacc')
('high', 'vhigh', '3', '2', 'med', 'low', 'unacc')
('high', 'vhigh', '3', '2', 'med', 'med', 'unacc')
('high', 'vhigh', '3', '2', 'med', 'high', 'unacc')
('high', 'vhigh', '3', '2', 'big', 'low', 'unacc')
('high', 'vhigh', '3', '2', 'big', 'med', 'unacc')
('high', 'vhigh', '3', '2', 'big', 'high', 'unacc')
('high', 'vhigh', '3', '4', 'small', 'low', 'unacc')
('high', 'vhigh', '3', '4', 'small', 'med', 'unacc')
('high', 'vhigh', '3', '4', 'small', 'high', 'unacc')
('high', 'vhigh', '3', '4', 'med', 'low', 'unacc')
('high', 'vhigh', '3', '4', 'med', 'med', 'unacc')
('high', 'vhigh', '3', '4', 'med', 'high', 'unacc')
('high', 'vhigh', '3', '4', 'big', 'low', 'unacc')
('high', 'vhigh', '3', '4', 'big', 'med', 'unacc')
('high', 'vhigh', '3', '4', 'big', 'high', 'unacc')
('high', 'vhigh', '3', 'more', 'small', 'low

In [52]:
mydb.update_data("car",{"doors":"8"},{"doors":"3"})

In [53]:
mydb.delete_table("car")

In [54]:
mydb.delete_database("cardataset")